# FabFit AltAir V3

## Imports

In [15]:
import time
import pandas as pd
import altair as alt
import pyodbc
import ipywidgets as widgets
from ipywidgets import IntProgress
from IPython.display import display, clear_output
from yaspin import yaspin, Spinner

import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

## Functions

### Database Functions

In [2]:
# Initiates DB connection - returns a "conn"
def initiate_conn(id_sid):

    # Initiate Connection to the db
    conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=sql2557-fm1s-in.amr.corp.intel.com, 3181;'
                          'Database=Fabfit_Prod;'
                          'UID=' + id_sid + ';'
                          'Trusted_Connection=yes;'
                          )
    return conn

# Returns eoh, decom, shortage table
def get_eds_rpt(conn, id_version):
    sql = """\
    EXEC lyt.pr_Report_Adj_CapPlan_EOH_Decommision_Shortage @VersionID = ?
    """
    # test = cursor.execute(sql, params)
    table_eoh_decom_short = pd.read_sql(sql, conn, params=[id_version])

    return table_eoh_decom_short

# Returns wif allocation table
def get_allocation(conn, id_version):
    sql = """\
    EXEC lyt.pr_Allocation_Get @VersionID = ?
    """
    # test = cursor.execute(sql, params)
    alloc_table = pd.read_sql(sql, conn, params=[id_version])

    alloc_table = alloc_table[alloc_table.Element == "WhatIf"]
    alloc_table = alloc_table.groupby(['Process']).sum().reset_index()

    return alloc_table

### Formating Functions

In [3]:
# Format date column
def format_date(df):
    # Add 15 to each date value
    df["date"] = df["date"] + "15"

    # Convert to datetime value
    df["date"] = pd.to_datetime(df["date"])

    # Set index to date
    df = df.set_index("date")

    return df


# Melt table
def melt_table(table, id_vars, value_vars, value_name):
    melted_table = table.melt(id_vars, var_name=value_vars, value_name=value_name).reset_index(drop=True)
    return melted_table


# Filter alloc report
def format_alloc_report(table_alloc, proc_list):
    # Filter usign the proc_list
    table_alloc = table_alloc[table_alloc["Process"].isin(proc_list)]
    return table_alloc


def format_eds_report(table_eds, proc_list):
    # Filter usign the proc_list
    table_eds = table_eds[table_eds["Process"].isin(proc_list)]

    # Filter "ToolType"
    table_eds = table_eds[table_eds["ToolType"] == "USC Tool"].fillna(0)

    return table_eds



### Make Functions

In [4]:

def get_source_data(version_id, cxn):
    # Get EDS and Alloc report
    eds_df = get_eds_rpt(cxn, version_id) # gets eoh, demo, shortage table
    alloc_df = get_allocation(cxn, version_id) # gets wif allocation

    return eds_df, alloc_df


def make_eds_report(eds_df, process):
    
    eds_proc = format_eds_report(eds_df, process)

    # Group By FA and sum
    eds_proc = eds_proc.groupby(['Type', 'Process']).sum().reset_index()

    eds_proc = melt_table(eds_proc, ['Process', 'Type'], 'date', 'tool_count')
    eds_proc = pd.pivot_table(eds_proc, index=['date', 'Process'], columns='Type', values='tool_count').reset_index()
    eds_proc = format_date(eds_proc)

    return eds_proc


def make_detailed_eds_report(eds_df, process):
    
    eds_proc = format_eds_report(eds_df, process)

    # Group By FA and sum
    eds_proc = eds_proc.groupby(['Type', 'Process', 'FuncArea']).sum().reset_index()
    
    eds_proc = melt_table(eds_proc, ['Process', 'Type', 'FuncArea'], 'date', 'tool_count')
    # eds_proc = pd.pivot_table(eds_proc, index=['date', 'Process'], columns='Type', values='tool_count').reset_index()
    eds_proc = format_date(eds_proc).reset_index()

    return eds_proc


def make_alloc_report(alloc_df, process):

    alloc_proc = format_alloc_report(alloc_df, process)
    alloc_proc = melt_table(alloc_proc, 'Process', 'date', 'wspw')
    alloc_proc = format_date(alloc_proc)

    return alloc_proc


def make_full(alloc_proc, eds_proc, proc_list):

    # Crosstab - Used, Short, Needed Bay Len Metrics
    full_df = pd.concat([alloc_proc, eds_proc],axis=1, sort=False)
    full_df.reset_index(inplace=True)
    full_df.fillna(0, inplace=True)

    # If shortage column exists get ratio and rate else fill ratio and rate with 0
    if 'Shortage' not in full_df.columns:
        full_df["Shortage"] = 0

    # Set ration, rate, and tool needed columns
    full_df["ratio"] = (full_df.Shortage / full_df.EOH) *100
    full_df["rate"] = full_df["Shortage"] - full_df["Shortage"].shift(1)
    full_df["tools_needed"] = full_df["EOH"] + full_df["Shortage"]

    # Remove duplicated columns
    full_df = full_df.loc[:,~full_df.columns.duplicated()]

    # Filter df using the proc list
    full_df = full_df[full_df["Process"].isin(proc_list)]
    return full_df



### Graphing Functions

In [5]:
def make_bar_graph(full_df, proc_list, version_id):

    # Makes shortage bar graph for first item in the proc list
    proc_num = proc_list[0]
    full_df = full_df[full_df.Process == proc_num]

    # Init graph object and set title
    base = alt.Chart(full_df, title="v" + str(version_id) + "-" + proc_num + " ramp").encode(
        alt.X('yearmonth(date):O', axis=alt.Axis(title=None))
    )

    # Create color
    color = 'steelblue' if proc_num == '1276' else 'purple'

    # Create bar graph
    bar = base.mark_bar(opacity=0.4).encode(
        alt.Y("wspw:Q",
            axis=alt.Axis(title=proc_num + ' WSPW', titleColor=color),
            scale=alt.Scale(domain=[0, 20000])),
                color=alt.condition(
            (alt.datum.rate > 0),  # If the year is 1810 this test returns True,
            alt.value('red'),     # which sets the bar orange.
            alt.value(color)   # And if it's not true it sets the bar steelblue.
        )
    )

    # Create dots on graph
    tick = base.mark_circle(
        color='red',
        size=40 * 0.9,  # controls width of tick.
    ).transform_filter(alt.datum.rate > 0).encode(
        alt.Y('Shortage',
            axis=alt.Axis(title='Shortage Total', titleColor='red'))
    )

    # Combine the graph
    my_graph = alt.layer(bar, tick).resolve_scale(
        y = 'independent'
    )

    # Display the graph
    my_graph.display()


def make_area_graph(full_df, proc_list, version_id):

    my_graph = alt.Chart(full_df).mark_area(opacity=0.6).encode(
        x="yearmonth(date):O",
        y="wspw:Q",
        color="Process:N",
        order=alt.Order("Process:N", sort='ascending')
    ).properties(width=1000)
    
    my_graph.display()

def make_line_graph(full_df, proc_list, version_id):

    for proc_num in proc_list:
        proc_df = full_df[full_df.Process == proc_num]

        # Set title
        title = f"Tools Needed vs EOH: {proc_num} - v{version_id}"

        # Make line graph
        my_graph = alt.Chart(proc_df, title=title).transform_fold(
            ['EOH', 'tools_needed'],
            as_=['type', 'num_tools']
        ).mark_line(opacity=0.6).encode(
            x="yearmonth(date):O",
            y=alt.Y("num_tools:Q", stack=None),
            color="type:N"
        ).properties(width=1000)

        my_graph.display()

def make_detailed_eds_graph(eds_df, proc_list, version_id):

    unique_fa = list(eds_df.FuncArea.unique())

    input_dropdown = alt.binding_select(options=unique_fa, name='FuncArea')
    selection = alt.selection_single(fields=['FuncArea'], bind=input_dropdown)

    for proc_num in proc_list:
        proc_df = eds_df[eds_df.Process == proc_num]

        # Set title
        title = f"Shortage: {proc_num} - v{version_id}"

        my_graph = alt.Chart(proc_df, title=title).mark_bar(opacity=0.6).encode(
            x="yearmonth(date):O",
            y=alt.Y("sum(tool_count):Q"),
            color="FuncArea:N",
            tooltip=['tool_count']
        ).interactive().properties(
            width=1000
        ).add_selection(
            selection
        ).transform_filter(
            selection
        )

        my_graph.display()

### Analysis Functions

In [6]:
# Function decorator:
@yaspin(text="Loading...")
def run_analysis(version_id, proc_list, cxn):
    # Get data from db
    eds_df, alloc_df = get_source_data(version_id, cxn) # same

    # Returns formatted eds and alloc reports
    eds_proc = make_eds_report(eds_df, proc_list)
    alloc_proc = make_alloc_report(alloc_df, proc_list)

    # Returns formatted eds and alloc reports
    d_eds_proc = make_detailed_eds_report(eds_df, proc_list)
    df_eds = fa_eds_report(d_eds_proc)

    # Returns full_df
    full_df = make_full(alloc_proc, eds_proc, proc_list)

    summary_metrics = full_df.iloc[-len(proc_list):][["Process","Shortage", "wspw"]]
    # print(f"Version {version_id} Summary:\n", summary_metrics)
    # print("\n\n")
    # print(f"Version {version_id} Summary:\n")
    display(f"Version {version_id} Summary:", summary_metrics.style.hide_index())
    print("\n\n")
    

    
    # Makes graphs
    make_area_graph(full_df, proc_list, version_id)
    make_line_graph(full_df, proc_list, version_id)
    make_detailed_eds_graph(df_eds, proc_list, version_id)

def fa_eds_report(table_eds):
    # Filter "ToolType"
    table_eds = table_eds[table_eds["Type"] == "Shortage"].fillna(0)
    
    # Group By FA and sum
    table_eds = table_eds.groupby(['date','Type', 'Process', 'FuncArea']).tool_count.sum().reset_index()

    return table_eds


## Visuals

In [7]:
idsid = widgets.Text(
    # value='AMR\idsid',
    placeholder='AMR\idsid',
    description='String:',
    disabled=False
)

version_id = widgets.IntText(
    value=7,
    description='VersionID:',
    disabled=False
)

process_tuple = widgets.SelectMultiple(
    options=["1270", "1272", "1274", "1276", "1278", "1280"],
    # value='macOS',
    # rows=10,
    description='Process:',
    disabled=False
)

In [8]:
display(idsid, version_id, process_tuple)

Text(value='', description='String:', placeholder='AMR\\idsid')

IntText(value=7, description='VersionID:')

SelectMultiple(description='Process:', options=('1270', '1272', '1274', '1276', '1278', '1280'), value=())

In [17]:
button = widgets.Button(description="Run Analysis")
output = widgets.Output()
label = widgets.Text('Ready')
display(label, button, output)

# Compose new spinners with custom frame sequence and interval value
sp = Spinner(["😸", "😹", "😺", "😻", "😼", "😽", "😾", "😿", "🙀"], 200)

# def loop_vzn_list(cxn):
#     label.value = 'Running...'
#     # Loop thorugh vzn_list
#     for item in vzn_list:
#         v_id = item[0]
#         proc_list = item[1]
#         run_analysis(v_id, proc_list, cxn)
        
# # Context manager:
# with yaspin().moon:
# label.value = 'Ready...'


# Function decorator:
@yaspin(sp, text="Loading...")
def on_button_clicked(b):
    with output:
        # Cleaer cell output
        clear_output()

        # Creates a list of tuples
        vzn_list = [(version_id.value, list(process_tuple.value))]
        
        # Get values
        vzn_id = vzn_list[0][0]
        proc_list = vzn_list[0][1]
        print("\n\n")
        print(f"The selected version is... {vzn_id} and process entered were {proc_list}", "\n\n")

        # Initiate DB connection
        cxn = initiate_conn(idsid.value)
        
        run_analysis(vzn_id, proc_list, cxn)

        # Call loop function
        # loop_vzn_list(cxn)
        
        # Change label output
        label.value = 'Done'


button.on_click(on_button_clicked)

Text(value='Ready')

Button(description='Run Analysis', style=ButtonStyle())

Output()